<a href="https://colab.research.google.com/github/KseniiaRyuma/notebooks/blob/main/ecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cohere pinecone-client datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1


In [ ]:
COHERE_KEY = "fXDGvwuPvSQnv8F6qGoUj8bfvG4YEvvtoAVrhf9z"
PINECONE_KEY = "8b7f687e-06e4-4d4c-b666-0bdac83c04d7"  # app.pinecone.io

## Create Embeddings

In [ ]:
import cohere
import numpy as np

co = cohere.Client(COHERE_KEY)

In [ ]:
import pandas as pd

products = pd.read_csv('/content/sample_product_descriptions.csv')

In [ ]:
products

,Product Description
0,Elevate your kitchen tasks with our premium dr...
1,Maximize your beauty routines with the dynamic...
2,Embrace the art of pet care with our exquisite...
3,"Step into the future with our blender, a game-..."
4,Experience unparalleled comfort with these fic...
...,...
495,Embrace the art of home improvement projects w...
496,Revolutionize your sports training with our cu...
497,Maximize your beauty routines with the dynamic...
498,Elevate your athletic activities with our prem...


In [ ]:
embeds = co.embed(
    texts=products['Product Description'].tolist(),
    model='embed-multilingual-v2.0',
    truncate='LEFT'
).embeddings

In [ ]:
shape = np.array(embeds).shape
shape

(500, 768)

## Storing the Embeddings

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_KEY)

index_name = 'products'

# connect to index
index = pc.Index(index_name)

Now we can begin populating the index with our embeddings. Pinecone expects us to provide a list of tuples in the format (id, vector, metadata), where the metadata field is an optional extra field where we can store anything we want in a dictionary format. For this example, we will store the original text of the embeddings.

While uploading our data, we will batch everything to avoid pushing too much data in one go.

In [ ]:
ids = [str(id) for id in products.index.tolist()]

batch_size = 16

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds))

for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

# let's view the index statistics
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.005,
 'namespaces': {'': {'vector_count': 500}},
 'total_vector_count': 500}

In [ ]:
# STAGE 1: Run query

# Губная помада
query = "Губная помада"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='embed-multilingual-v2.0',
    truncate='LEFT'
).embeddings

print(np.array(xq).shape)

# query, returning the top 10 most similar results
res = index.query(vector=xq, top_k=5, include_metadata=True)
res

(1, 768)


{'matches': [{'id': '140', 'score': 0.884399, 'values': []},
             {'id': '282', 'score': 0.883328199, 'values': []},
             {'id': '468', 'score': 0.880855203, 'values': []},
             {'id': '200', 'score': 0.876894951, 'values': []},
             {'id': '306', 'score': 0.875544429, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [42]:
# Get the product description for each STAGE 1 result
docs = []
for m in res['matches']:
   docs.append(products.iloc[int(m['id'])]['Product Description'])



In [43]:
docs

['Revolutionize your reading enjoyment with our cutting-edge fiction novel. Experience innovation at its best, offering efficiency and versatility.',
 'Transform your reading enjoyment with the latest fiction novel. Featuring advanced technology, it sets new standards in the industry.',
 'Transform your reading enjoyment with the latest fiction novel. Featuring advanced technology, it sets new standards in the industry.']

In [44]:


import csv
import json

results = []

with open("/content/random_product_searches.csv", 'r') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)

    for row in csvreader:

      d = {'query': row[0], 'relevant_passages': []}
      xq = co.embed(
        texts=[row[0]],
        model='embed-multilingual-v2.0',
        truncate='LEFT'
         ).embeddings

      # query, returning the top 10 most similar results
      res = index.query(vector=xq, top_k=3, include_metadata=True)
      for m in res['matches']:
            d['relevant_passages'].append(products.iloc[int(m['id'])]['Product Description'])

      results.append(d)

#Convert to list json

with open('/content/new_file.jsonl', 'w') as f:
  for ddict in results:
      jout = json.dumps(ddict) + '\n'
      f.write(jout)

In [ ]:
# STAGE 1 results contain irrelevant results (e.g., products related to men)
docs

['Revolutionize your playtime with our cutting-edge lipstick. Experience innovation at its best, offering efficiency and versatility.',
 'Revolutionize your daily life with our cutting-edge lipstick. Experience innovation at its best, offering efficiency and versatility.',
 'Revolutionize your home improvement projects with our cutting-edge lipstick. Experience innovation at its best, offering efficiency and versatility.',
 'Elevate your playtime with our premium lipstick. Crafted with excellence, it provides superior quality and unmatched durability.',
 'Transform your home improvement projects with the latest lipstick. Featuring advanced technology, it sets new standards in the industry.']

In [ ]:
# query, [docs]

In [ ]:
# STAGE 2: Rerank the result
rerank_hits = co.rerank(query=query, documents=docs, top_n=3, model='rerank-multilingual-v2.0')

In [ ]:
# As you can see, the rerank model applied semantic search and only kept women's results
rerank_hits

[RerankResult<document['text']: Revolutionize your daily life with our cutting-edge lipstick. Experience innovation at its best, offering efficiency and versatility., index: 1, relevance_score: 0.70801073>, RerankResult<document['text']: Elevate your playtime with our premium lipstick. Crafted with excellence, it provides superior quality and unmatched durability., index: 3, relevance_score: 0.66963685>, RerankResult<document['text']: Revolutionize your playtime with our cutting-edge lipstick. Experience innovation at its best, offering efficiency and versatility., index: 0, relevance_score: 0.53740555>]

In [51]:
with open('/content/productlist.jsonl', 'r') as f:
  import json

  data = json.load(f)


  rows = []

  for line in data["data"]:
    line['relevant_passages'] = [line.pop('relevant_passage')]
    rows.append(json.dumps(line))

with open('/content/new_file.jsonl', 'w') as f:
  f.write('\n'.join(rows))



In [ ]:
from google.colab import drive
drive.mount('/content/drive')